In [4]:

import sys
sys.path.append('thirdparty/AdaptiveWingLoss')
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import util.utils as util
from scipy.signal import savgol_filter

from src.approaches.train_audio2landmark import Audio2landmark_model

In [5]:

default_head_name = 'dali'
ADD_NAIVE_EYE = True
CLOSE_INPUT_FACE_MOUTH = False

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name))
parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')

parser.add_argument('--load_AUTOVC_name', type=str, default='examples/ckpt/ckpt_autovc.pth')
parser.add_argument('--load_a2l_G_name', type=str, default='examples/ckpt/ckpt_speaker_branch.pth')
parser.add_argument('--load_a2l_C_name', type=str, default='examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
parser.add_argument('--load_G_name', type=str, default='examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c

parser.add_argument('--amp_lip_x', type=float, default=2.)
parser.add_argument('--amp_lip_y', type=float, default=2.)
parser.add_argument('--amp_pos', type=float, default=.5)
parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
parser.add_argument('--add_audio_in', default=False, action='store_true')
parser.add_argument('--comb_fan_awing', default=False, action='store_true')
parser.add_argument('--output_folder', type=str, default='examples')

parser.add_argument('--test_end2end', default=True, action='store_true')
parser.add_argument('--dump_dir', type=str, default='', help='')
parser.add_argument('--pos_dim', default=7, type=int)
parser.add_argument('--use_prior_net', default=True, action='store_true')
parser.add_argument('--transformer_d_model', default=32, type=int)
parser.add_argument('--transformer_N', default=2, type=int)
parser.add_argument('--transformer_heads', default=2, type=int)
parser.add_argument('--spk_emb_enc_size', default=16, type=int)
parser.add_argument('--init_content_encoder', type=str, default='')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
parser.add_argument('--write', default=False, action='store_true')
parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
parser.add_argument('--emb_coef', default=3.0, type=float)
parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
parser.add_argument('--use_11spk_only', default=False, action='store_true')

opt_parser = parser.parse_args()


usage: ipykernel_launcher.py [-h] [--jpg JPG] [--close_input_face_mouth] [--load_AUTOVC_name LOAD_AUTOVC_NAME]
                             [--load_a2l_G_name LOAD_A2L_G_NAME] [--load_a2l_C_name LOAD_A2L_C_NAME]
                             [--load_G_name LOAD_G_NAME] [--amp_lip_x AMP_LIP_X] [--amp_lip_y AMP_LIP_Y]
                             [--amp_pos AMP_POS]
                             [--reuse_train_emb_list REUSE_TRAIN_EMB_LIST [REUSE_TRAIN_EMB_LIST ...]] [--add_audio_in]
                             [--comb_fan_awing] [--output_folder OUTPUT_FOLDER] [--test_end2end] [--dump_dir DUMP_DIR]
                             [--pos_dim POS_DIM] [--use_prior_net] [--transformer_d_model TRANSFORMER_D_MODEL]
                             [--transformer_N TRANSFORMER_N] [--transformer_heads TRANSFORMER_HEADS]
                             [--spk_emb_enc_size SPK_EMB_ENC_SIZE] [--init_content_encoder INIT_CONTENT_ENCODER]
                             [--lr LR] [--reg_lr REG_LR] [--write] [--s

SystemExit: 2

In [13]:
img1 = r'statue1.jpg'
mout = True
''' STEP 1: preprocess input single image '''
print('hi',img1)
img =cv2.imread('examples/' + img1)
predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, device='cuda', flip_input=True)
shapes = predictor.get_landmarks(img)
if (not shapes or len(shapes) != 1):
    print('Cannot detect face landmarks. Exit.')
    exit(-1)
shape_3d = shapes[0]

if(mout):
    util.close_input_face_mouth(shape_3d)

hi statue1.jpg


In [14]:
np.array(shapes).shape

(1, 68, 3)